In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


2025-08-19 15:01:03.821319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755608463.843521   14425 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755608463.850853   14425 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755608463.865919   14425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755608463.865939   14425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755608463.865941   14425 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Importy

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


2025-09-16 23:42:25.085626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758058945.116370  281354 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758058945.123896  281354 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758058945.142863  281354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758058945.142887  281354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758058945.142889  281354 computation_placer.cc:177] computation placer alr

## Wczytanie modelu

In [2]:
model = load_model('../../model/2025-09-16/cnn_1d_model.keras')

I0000 00:00:1758058947.843137  281354 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 176 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1
I0000 00:00:1758058947.864490  281354 cuda_executor.cc:479] failed to allocate 176.62MiB (185204736 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory


## Wczytanie etykiet z pliku tekstowego


In [3]:
with open('../../model/2025-07-14/labels.txt', 'r') as f:
    label_names = [line.strip() for line in f.readlines()]

## MediaPipe inicjalizacja

In [4]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1758058951.528015  281451 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758058951.550209  281451 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Bufor sekwencji

In [5]:
sequence = []
SEQ_LENGTH = 30
THRESHOLD = 0.8

## Normalizacja względem nadgarstka pierwszej klatki

In [6]:
def normalize_sequence(sequence_array):
    sequence_array = np.array(sequence_array)
    origin = sequence_array[0][0]  # Nadgarstek 1. klatki
    return sequence_array - origin

## Ekstrakcja 21 punktów prawej dłoni

In [7]:
def extract_hand_landmarks(hand_landmarks):
    if hand_landmarks:
        return np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
    return None


# Kamera na żywo

In [2]:
# Kamera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Rysowanie dłoni
            mp_drawing.draw_landmarks(frame,
                                      hand_landmarks,
                                      mp_hands.HAND_CONNECTIONS)

            # Ekstrakcja landmarków (musisz mieć własną funkcję extract_hand_landmarks dostosowaną do mp_hands)
            landmarks = extract_hand_landmarks(hand_landmarks)

            if landmarks is not None:
                sequence.append(landmarks)

                if len(sequence) == SEQ_LENGTH:
                    norm_seq = normalize_sequence(sequence)
                    input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)

                    pred = model.predict(input_seq)[0]
                    max_prob = np.max(pred)
                    label = label_names[np.argmax(pred)]

                    if max_prob > THRESHOLD:
                        cv2.putText(frame, f'{label} ({max_prob:.2f})', (10, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    sequence.pop(0)
    else:
        sequence = []

    cv2.imshow('Real-Time Sign Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

# Kamera na żywo z cooldown'em

## Parametry

In [8]:
sequence = []
recognized_text = ""
last_prediction_time = 0
cooldown_seconds = 3
SEQ_LENGTH = 30
THRESHOLD = 0.9

In [9]:
import cv2
import mediapipe as mp
import numpy as np

# Inicjalizacja MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7)

# Kamera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Rysowanie dłoni
            mp_drawing.draw_landmarks(frame,
                                      hand_landmarks,
                                      mp_hands.HAND_CONNECTIONS)

            # Ekstrakcja landmarków (musisz mieć własną funkcję extract_hand_landmarks dostosowaną do mp_hands)
            landmarks = extract_hand_landmarks(hand_landmarks)

            if landmarks is not None:
                sequence.append(landmarks)

                if len(sequence) == SEQ_LENGTH:
                    norm_seq = normalize_sequence(sequence)
                    input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)

                    pred = model.predict(input_seq)[0]
                    max_prob = np.max(pred)
                    label = label_names[np.argmax(pred)]

                    if max_prob > THRESHOLD:
                        cv2.putText(frame, f'{label} ({max_prob:.2f})', (10, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    sequence.pop(0)
    else:
        sequence = []

    cv2.imshow('Real-Time Sign Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1758058965.117947  281466 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758058965.140228  281466 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758058969.182394  281470 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
I0000 00:00:1758058972.335467  281423 service.cc:152] XLA service 0x7f4808003c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758058972.335489  281423 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2025-09-16 23:42:52.342600: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 740ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


I0000 00:00:1758058972.994471  281423 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
